## 프로젝트: 멋진 작사가 만들기

### 라이브러리 로드

In [1]:
import glob #glob모듈의 glob함수는 사용자가 제시한 조건에 맞는 파일명을 리스트형식으로 반환
import tensorflow as tf
import os, re

####  Step 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv("HOME")+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


#### Step3.데이터 정제

In [3]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()

def preprocess_sentence(sentence):
    #일단 모두 소문자로 변환하고 양쪽 공백을 지웁니다.
    sentence = sentence.lower().strip()   
    tokens = sentence.split()
   
    # 아래 특수문자 기호 양쪽에 공백을 추가합니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
     # 공백이 많을 수 있는 부분에는 하나의 공백으로 통일합니다.
    sentence = re.sub(r'[" "]+', " ", sentence) 
     # ""안에 들어가있는 기호들 외에 공백으로 바꿉니다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    #다시 양쪽 공백을 지웁니다.
    sentence = sentence.strip()
    
    # 문장 시작과 끝에 start와 end 를 추가합니다.
    sentence = '<start> ' + sentence + ' <end>'   
    
    return tokens, sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

(['this', '@_is', ';;;sample', 'sentence.'], '<start> this is sample sentence . <end>')


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    tokens, preprocessed = preprocess_sentence(sentence)

    if len(tokens) <= 15: 
        corpus.append(preprocessed)
    
corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> nothin , don t mean nothin hon if it ain t free , no no <end>',
 '<start> and , feelin good was easy , lord , when he sang the blues <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>']

토큰화 ( 문자에 숫자 매핑 )

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words= 7000,
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    
    return tensor, tokenizer

In [6]:
tensor, tokenizer = tokenize(corpus)

[[   2 4134 1770 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  838    1 ...    0    0    0]
 ...
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f18e9c96d00>


In [7]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]


#### 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val,  = train_test_split(src_input, 
                                                         tgt_input,
                                                         test_size=0.2,
                                                         shuffle=True,
                                                         random_state = 34)
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (134685, 32)
Target Train:  (134685, 32)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)        
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear(out)
        
        return out
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)


In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 31,000,665
Trainable params: 31,000,665
Non-trainable params: 0
_________________________________________________________________


####  Step 5.인공지능 만들기

In [12]:
from tensorflow.python.keras.callbacks import TensorBoard 
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30, batch_size=64, callbacks=[tensorboard])

Epoch 1/30
657/657 [==============================] - 277s 413ms/step - loss: 1.6344
Epoch 2/30
657/657 [==============================] - 279s 424ms/step - loss: 1.4371
Epoch 3/30
657/657 [==============================] - 280s 426ms/step - loss: 1.3466
Epoch 4/30
657/657 [==============================] - 280s 426ms/step - loss: 1.2893
Epoch 5/30
657/657 [==============================] - 280s 426ms/step - loss: 1.2422
Epoch 6/30
657/657 [==============================] - 280s 425ms/step - loss: 1.2022
Epoch 7/30
657/657 [==============================] - 280s 425ms/step - loss: 1.1636
Epoch 8/30
657/657 [==============================] - 280s 425ms/step - loss: 1.1278
Epoch 9/30
657/657 [==============================] - 280s 426ms/step - loss: 1.0937
Epoch 10/30
657/657 [==============================] - 280s 426ms/step - loss: 1.0604
Epoch 11/30
657/657 [==============================] - 280s 426ms/step - loss: 1.0275
Epoch 12/30
657/657 [==============================] - 280s 426

In [15]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

#### 모델 테스트

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you so much <end> '

## 회고

- EmbeddingLayer를 통해서 문자를 숫자함수의 형태로 변환하여 모델링할 수 있도록 변환함을 알 수 있었음
- RNN의 단점 Weight을 하나만 가지고 모델링을하여 backpropagation시 vanishing gradient, exploding gradient 발생가능
- RNN의 단점을 보완하기 위해 LSTM은 forget gate, input gate, output gate등을 통하여 장기기억/단기기억 관리
- 앞/뒤 단어의 연계관계를 forget gate통해서 관리하고 tanh(탄젠트하이퍼볼릭)통하여 활성화여부 결정
- LSTM 레이어를 하나 더 (총 3개)로 층을 더 두어 테스트 하여 초반 loss가 1.6에서 시작하여 좀 더 나은 loss진입을 보여줌